<a href="https://colab.research.google.com/github/ameyas1/FakeNewClassification/blob/master/FakeNewsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install trax==1.3.1

     |████████████████████████████████| 348kB 4.6MB/s 
     |████████████████████████████████| 163kB 14.4MB/s 
     |████████████████████████████████| 2.6MB 14.2MB/s 
     |████████████████████████████████| 1.5MB 40.8MB/s 
     |████████████████████████████████| 307kB 48.4MB/s 
     |████████████████████████████████| 3.5MB 47.7MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 890kB 51.3MB/s 
     |████████████████████████████████| 1.1MB 48.4MB/s 
     |████████████████████████████████| 655kB 50.6MB/s 
     |████████████████████████████████| 5.3MB 52.9MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 983kB 53.6MB/s 
     |████████████████████████████████| 358kB 45.5MB/s 
     |████████████████████████████████| 368kB 48.4MB/s 
     |████████████████████████████████| 194kB 52.9MB/s 
     |████████████████████████████████| 3.0MB 31.3MB/s 
     |████████████████████████████████| 890kB 48.3MB

In [2]:
import pandas as pd
import random as rnd
import trax
trax.supervised.trainer_lib.init_random_number_generators(31)
import trax.fastmath.numpy as fastnp
from trax import layers as tl

INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 


In [3]:
!pip install plotly
!pip install --upgrade nbformat
!pip install nltk
!pip install spacy # spaCy is an open-source software library for advanced natural language processing
!pip install WordCloud
!pip install gensim # Gensim is an open-source library for unsupervised topic modeling and natural language processing
import nltk
nltk.download('punkt')

# import tensorflow as tf
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
# import keras
# from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
# from tensorflow.keras.models import Model
# from jupyterthemes import jtplot
# jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False) 
# setting the style of the notebook to be monokai theme  
# this line of code is important to ensure that we are able to see the x and y axes clearly
# If you don't run this code line, you will notice that the xlabel and ylabel on any plot is black on black and it will be hard to see them. 


Requirement already up-to-date: nbformat in /usr/local/lib/python3.6/dist-packages (5.0.7)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from sklearn.model_selection import train_test_split

In [9]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [10]:
!kaggle competitions download -c fake-news

 53% 5.00M/9.42M [00:00<00:00, 26.4MB/s]
100% 9.42M/9.42M [00:00<00:00, 37.3MB/s]
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 38.5MB/s]
 89% 33.0M/37.0M [00:00<00:00, 12.8MB/s]
100% 37.0M/37.0M [00:00<00:00, 47.3MB/s]


In [11]:
!unzip /content/train.csv.zip
!unzip /content/test.csv.zip

Archive:  /content/train.csv.zip
  inflating: train.csv               
Archive:  /content/test.csv.zip
  inflating: test.csv                


In [12]:
df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [13]:
df=df[["text","label"]]
# test_df=test_df[["text"]]

In [14]:
df.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
df.isna().sum()

text     39
label     0
dtype: int64

In [16]:
df.dropna(inplace=True)

In [17]:
all_pos = df[df.label==1]
all_neg = df[df.label==0]

In [18]:
len(all_pos),len(all_neg)

(10374, 10387)

In [19]:
all_pos_text = all_pos.text.to_list()
all_neg_text = all_neg.text.to_list()

In [20]:
train_pos, val_pos = train_test_split(all_pos_text, test_size=0.13)
train_neg, val_neg = train_test_split(all_neg_text, test_size=0.13)

In [21]:
train_x = train_pos + train_neg 
val_x  = val_pos + val_neg

train_y = fastnp.append(fastnp.ones(len(train_pos)), fastnp.zeros(len(train_neg)))
val_y  = fastnp.append(fastnp.ones(len(val_pos)), fastnp.zeros(len(val_neg)))
print(f"length of train_x {len(train_x)}")
print(f"length of val_x {len(val_x)}")

length of train_x 18061
length of val_x 2700


In [22]:
# download stopwords
nltk.download("stopwords")
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [23]:
# Remove stopwords and remove words with 2 or less characters
def preprocess(text):
    result = []
    text = re.sub(r'#', '', text)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words and token not in string.punctuation:
            result.append(token)
            
    return result

In [24]:
# Try out function that processes tweets
print("original tweet at training position 0")
print(train_pos[0])

print("Tweet at training position 0 after processing:")
preprocess(train_pos[0])

original tweet at training position 0
VICE News Fri, November 18, 2016 10:38am URL: Embed: 
With two avowed supporters of the anti-abortion movement set to take the White House in just 63 days, Planned Parenthood will have to prepare for deep cuts and open hostility. 
Caroline Modarressy-Tehrani went to Washington D.C. where Planned Parenthood’s leadership met behind closed doors to mount its counter-attack. 
Watch VICE News Tonight weeknights at 7:30 on HBO. 
Subscribe to VICE News here: http://bit.ly/Subscribe-to-VICE-News 
Check out VICE News for more: http://vicenews.com 
Follow VICE News here: 
Tweet at training position 0 after processing:


['vice',
 'news',
 'november',
 'embed',
 'avowed',
 'supporters',
 'anti',
 'abortion',
 'movement',
 'white',
 'house',
 'days',
 'planned',
 'parenthood',
 'prepare',
 'deep',
 'cuts',
 'open',
 'hostility',
 'caroline',
 'modarressy',
 'tehrani',
 'went',
 'washington',
 'planned',
 'parenthood',
 'leadership',
 'closed',
 'doors',
 'mount',
 'counter',
 'attack',
 'watch',
 'vice',
 'news',
 'tonight',
 'weeknights',
 'subscribe',
 'vice',
 'news',
 'check',
 'vice',
 'news',
 'follow',
 'vice',
 'news']

In [25]:
# Build the vocabulary
# Unit Test Note - There is no test set here only train/val

# Include special tokens 
# started with pad, end of line and unk tokens
Vocab = {'__PAD__': 0, '__</e>__': 1, '__UNK__': 2} 
# Note that we build vocab using training data
for x in train_x: 
    processed_tweet = preprocess(x)
    for word in processed_tweet:
        if word not in Vocab: 
            Vocab[word] = len(Vocab)
    
print("Total words in vocab are",len(Vocab))


Total words in vocab are 154878


In [26]:
def text_to_tensor(text, vocab_dict, unk_token='__UNK__', verbose=False):
    '''
    Input: 
        tweet - A string containing a tweet
        vocab_dict - The words dictionary
        unk_token - The special string for unknown tokens
        verbose - Print info durign runtime
    Output:
        tensor_l - A python list with
        
    '''   
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    # Process the tweet into a list of words
    # where only important words are kept (stop words removed)
    word_l = preprocess(text)
    
    if verbose:
        print("List of words from the processed tweet:")
        print(word_l)
        
    # Initialize the list that will contain the unique integer IDs of each word
    tensor_l = []
    
    # Get the unique integer ID of the __UNK__ token
    unk_ID = vocab_dict[unk_token]
    
    if verbose:
        print(f"The unique integer ID for the unk_token is {unk_ID}")
        
    # for each word in the list:
    for word in word_l:
        
        # Get the unique integer ID.
        # If the word doesn't exist in the vocab dictionary,
        # use the unique ID for __UNK__ instead.
        word_ID = vocab_dict.get(word,unk_ID)
    ### END CODE HERE ###
        
        # Append the unique integer ID to the tensor list.
        tensor_l.append(word_ID) 
    
    return tensor_l

In [27]:
print("Actual tweet is\n", val_pos[0])
print("\nTensor of tweet:\n", text_to_tensor(val_pos[0], vocab_dict=Vocab))

Actual tweet is
 Trump Mistakes Ex-Marine Black Supporter For Protester, Calls Him A ‘Thug’ (TWEET/VIDEO) ‹ › Trump Says Alec Baldwin’s Impression Of Him Is Unfair: ‘He’s Portraying Someone Mean And Nasty’ (VIDEO) By Andrew Bradford on October 29, 2016 Subscribe 
Donald Trump doesn’t have a sense of humor. While he may smile from time to time, it always seems forced, as if he’s afraid his face will crack if he smiles too broadly. And now he’s whining yet again about Alec Baldwin’s brilliant portrayal of him on Saturday Night Live. 
As part of an interview with AJ Calloway of Extra , Trump had this to say about the way Baldwin plays him: “Well, I think I’m a much nicer guy than he’s portraying. He’s portraying someone who’s very mean and nasty, and I’m not mean and nasty. I think I’m a much nicer person than he’s portraying, so I think it’s an inaccurate portrayal of me.” 
Clearly, in addition to Trump not having a sense of humor, he also is deficient in the self-awareness department. 


In [28]:
def data_generator(data_pos, data_neg, batch_size, loop, vocab_dict, shuffle=False):
    '''
    Input: 
        data_pos - Set of posstive examples
        data_neg - Set of negative examples
        batch_size - number of samples per batch. Must be even
        loop - True or False
        vocab_dict - The words dictionary
        shuffle - Shuffle the data order
    Yield:
        inputs - Subset of positive and negative examples
        targets - The corresponding labels for the subset
        example_weights - An array specifying the importance of each example
        
    '''     
### START GIVEN CODE ###
    # make sure the batch size is an even number
    # to allow an equal number of positive and negative samples
    assert batch_size % 2 == 0
    
    # Number of positive examples in each batch is half of the batch size
    # same with number of negative examples in each batch
    n_to_take = batch_size // 2
    
    # Use pos_index to walk through the data_pos array
    # same with neg_index and data_neg
    pos_index = 0
    neg_index = 0
    
    len_data_pos = len(data_pos)
    len_data_neg = len(data_neg)
    
    # Get and array with the data indexes
    pos_index_lines = list(range(len_data_pos))
    neg_index_lines = list(range(len_data_neg))
    
    # shuffle lines if shuffle is set to True
    if shuffle:
        rnd.shuffle(pos_index_lines)
        rnd.shuffle(neg_index_lines)
        
    stop = False
    
    # Loop indefinitely
    while not stop:  
        
        # create a batch with positive and negative examples
        batch = []
        
        # First part: Pack n_to_take positive examples
        
        # Start from pos_index and increment i up to n_to_take
        for i in range(n_to_take):
                    
            # If the positive index goes past the positive dataset lenght,
            if pos_index >= len_data_pos: 
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;
                
                # If user wants to keep re-using the data, reset the index
                pos_index = 0
                
                if shuffle:
                    # Shuffle the index of the positive sample
                    rnd.shuffle(pos_index_lines)
                    
            
            text = data_pos[pos_index_lines[pos_index]]
            
            # convert the tweet into tensors of integers representing the processed words
            tensor = text_to_tensor(text, vocab_dict)

            tensor = tensor[:1024]
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment pos_index by one
            pos_index = pos_index + 1

### END GIVEN CODE ###
            
### START CODE HERE (Replace instances of 'None' with your code) ###

        # Second part: Pack n_to_take negative examples
    
        # Using the same batch list, start from neg_index and increment i up to n_to_take
        for i in range(n_to_take):
            
            # If the negative index goes past the negative dataset length,
            if neg_index >= len_data_neg:
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;
                    
                # If user wants to keep re-using the data, reset the index
                neg_index = 0
                
                if shuffle:
                    # Shuffle the index of the negative sample
                    rnd.shuffle(neg_index_lines)
            # get the tweet as neg_index
            text = data_neg[neg_index_lines[neg_index]]
            
            # convert the tweet into tensors of integers representing the processed words
            tensor = text_to_tensor(text, vocab_dict)

            tensor = tensor[:1024]
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment neg_index by one
            neg_index += 1

### END CODE HERE ###        

### START GIVEN CODE ###
        if stop:
            break;

        # Update the start index for positive data 
        # so that it's n_to_take positions after the current pos_index
        # pos_index += n_to_take
        
        # Update the start index for negative data 
        # so that it's n_to_take positions after the current neg_index
        # neg_index += n_to_take
        
        # Get the max tweet length (the length of the longest tweet) 
        # (you will pad all shorter tweets to have this length)
        max_len = max([len(t) for t in batch]) 
        
        
        # Initialize the input_l, which will 
        # store the padded versions of the tensors
        tensor_pad_l = []
        # Pad shorter tweets with zeros
        for tensor in batch:
### END GIVEN CODE ###

### START CODE HERE (Replace instances of 'None' with your code) ###
            # Get the number of positions to pad for this tensor so that it will be max_len long
            n_pad = max_len - len(tensor)
            
            # Generate a list of zeros, with length n_pad
            pad_l = [0]*n_pad
            
            # concatenate the tensor and the list of padded zeros
            tensor_pad = tensor + pad_l
            
            # append the padded tensor to the list of padded tensors
            tensor_pad_l.append(tensor_pad)

        # convert the list of padded tensors to a numpy array
        # and store this as the model inputs
        inputs = fastnp.array(tensor_pad_l)
  
        # Generate the list of targets for the positive examples (a list of ones)
        # The length is the number of positive examples in the batch
        target_pos = [1] * n_to_take
        
        # Generate the list of targets for the negative examples (a list of zeros)
        # The length is the number of negative examples in the batch
        target_neg = [0] * n_to_take
        
        # Concatenate the positve and negative targets
        target_l = target_pos + target_neg
        
        # Convert the target list into a numpy array
        targets = fastnp.array(target_l)

        # Example weights: Treat all examples equally importantly.
        example_weights = [1]*2*n_to_take

        inds = list(range(2*n_to_take))
        rnd.shuffle(inds)

        inputs = inputs[inds]
        targets = targets[inds]
        

### END CODE HERE ###

### GIVEN CODE ###
        # note we use yield and not return
        yield inputs, targets, fastnp.array(example_weights)

In [29]:
rnd.seed(30) 

# Create the training data generator
def train_generator(batch_size, shuffle = False):
    return data_generator(train_pos, train_neg, batch_size, True, Vocab, shuffle)

# Create the validation data generator
def val_generator(batch_size, shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, True, Vocab, shuffle)

# Create the validation data generator
def test_generator(batch_size, shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, False, Vocab, shuffle)

# Get a batch from the train_generator and inspect.
inputs, targets, example_weights = next(train_generator(4, shuffle=True))

# this will print a list of 4 tensors padded with zeros
print(inputs.shape)
print(f'Inputs: {inputs}')
print(f'Targets: {targets}')
print(f'Example Weights: {example_weights}')

(4, 923)
Inputs: [[114664 137765   1252 ...      0      0      0]
 [ 36214   5486   2245 ...    119    497   1113]
 [  2530    573   3499 ...      0      0      0]
 [    70     13   3420 ...      0      0      0]]
Targets: [0 1 1 0]
Example Weights: [1 1 1 1]


In [30]:
# Create a data generator for training data,
# which produces batches of size 4 (for tensors and their respective targets)
tmp_data_gen = test_generator(batch_size = 4)

# Call the data generator to get one batch and its targets
tmp_inputs, tmp_targets, tmp_example_weights = next(tmp_data_gen)

print(f"The inputs shape is {tmp_inputs.shape}")
for i,t in enumerate(tmp_inputs):
    print(f"input tensor: {t}; target {tmp_targets[i]}; example weights {tmp_example_weights[i]}")

The inputs shape is (4, 645)
input tensor: [96892     2   178  1113  3656   231 39342   493  1874 39342  3520  1792
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0  

In [31]:
def classifier(vocab_size=len(Vocab), embedding_dim=128, output_dim=2, mode='train'):

    # embed_layer = tl.Embedding(
    #     vocab_size=vocab_size, # Size of the vocabulary
    #     d_feature=embedding_dim)  # Embedding dimension
    
    # # Create a mean layer, to create an "average" word embedding
    # mean_layer = tl.Mean(axis=1)
    
    # # Create a dense layer, one unit for each output
    # dense_output_layer = tl.Dense(n_units = output_dim)

    
    # # Create the log softmax layer (no parameters needed)
    # log_softmax_layer = tl.LogSoftmax()

    model = trax.models.transformer.TransformerEncoder(vocab_size, n_classes=output_dim, d_model=embedding_dim, d_ff=2048, n_layers=1, n_heads=8, dropout=0.1, dropout_shared_axes=None, max_len=1024, mode=mode)
    # model = trax.models.research.bert.BERTClassifierHead(2)  
    # model = tl.Serial(
       
    #   embed_layer, # embedding layer
    #   mean_layer, # mean layer
    #   dense_output_layer, # dense output layer 
    #   log_softmax_layer # log softmax layer
    # )
    return model

In [32]:
tmp_model = classifier()

In [33]:
print(type(tmp_model))
display(tmp_model)

<class 'trax.layers.combinators.Serial'>


Serial[
  Branch_out2[
    [Embedding_154878_128, Dropout, PositionalEncoding]
    PaddingMask(0)
  ]
  Serial_in2_out2[
    Branch_in2_out3[
      None
      Serial_in2_out2[
        LayerNorm
        Serial_in2_out2[
          Dup_out2
          Dup_out2
          Serial_in4_out2[
            Parallel_in3_out3[
              Dense_128
              Dense_128
              Dense_128
            ]
            PureAttention_in4_out2
            Dense_128
          ]
        ]
        Dropout
      ]
    ]
    Add_in2
  ]
  Serial[
    Branch_out2[
      None
      Serial[
        LayerNorm
        Dense_2048
        Relu
        Dropout
        Dense_128
        Dropout
      ]
    ]
    Add_in2
  ]
  Select[0]_in2
  LayerNorm
  Mean
  Dense_2
  LogSoftmax
]

In [34]:
from trax.supervised import training

batch_size = 16
rnd.seed(271)

lr_schedule = trax.supervised.lr_schedules.warmup_and_rsqrt_decay(1300,0.001)

train_task = training.TrainTask(
    labeled_data=train_generator(batch_size=batch_size, shuffle=True),
    loss_layer=tl.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adam(0.001),
    n_steps_per_checkpoint=10,
    lr_schedule=lr_schedule
)

eval_task = training.EvalTask(
    labeled_data=val_generator(batch_size=batch_size, shuffle=True),
    metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
    n_eval_batches = 32
)

model = classifier()

In [35]:
import os
output_dir = '/content/'
output_dir_expand = os.path.expanduser(output_dir)
print(output_dir_expand)

/content/


In [36]:
def train_model(classifier, train_task, eval_task, n_steps, output_dir):
    '''
    Input: 
        classifier - the model you are building
        train_task - Training task
        eval_task - Evaluation task
        n_steps - the evaluation steps
        output_dir - folder to save your files
    Output:
        trainer -  trax trainer
    '''
### START CODE HERE (Replace instances of 'None' with your code) ###
    training_loop = training.Loop(
                                classifier, # The learning model
                                train_task, # The training task
                                eval_task = eval_task, # The evaluation task
                                output_dir = output_dir) # The output directory

    training_loop.run(n_steps = n_steps)
### END CODE HERE ###

    # Return the training_loop, since it has the model.
    return training_loop

In [37]:
training_loop = train_model(model, train_task, eval_task, 3900, output_dir_expand)

Step      1: train CrossEntropyLoss |  0.75910896
Step      1: eval  CrossEntropyLoss |  0.77631182
Step      1: eval          Accuracy |  0.50000000
Step     10: train CrossEntropyLoss |  0.76132399
Step     10: eval  CrossEntropyLoss |  0.72381587
Step     10: eval          Accuracy |  0.49804688
Step     20: train CrossEntropyLoss |  0.68720669
Step     20: eval  CrossEntropyLoss |  0.65392923
Step     20: eval          Accuracy |  0.62695312
Step     30: train CrossEntropyLoss |  0.68213230
Step     30: eval  CrossEntropyLoss |  0.66771900
Step     30: eval          Accuracy |  0.58984375
Step     40: train CrossEntropyLoss |  0.66077757
Step     40: eval  CrossEntropyLoss |  0.64561101
Step     40: eval          Accuracy |  0.64843750
Step     50: train CrossEntropyLoss |  0.68251908
Step     50: eval  CrossEntropyLoss |  0.63639805
Step     50: eval          Accuracy |  0.66992188
Step     60: train CrossEntropyLoss |  0.62826985
Step     60: eval  CrossEntropyLoss |  0.62661697


In [38]:
# Create a generator object
tmp_train_generator = train_generator(16)

# get one batch
tmp_batch = next(tmp_train_generator)

# Position 0 has the model inputs (tweets as tensors)
# position 1 has the targets (the actual labels)
tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

print(f"The batch is a tuple of length {len(tmp_batch)} because position 0 contains the tweets, and position 1 contains the targets.") 
print(f"The shape of the tweet tensors is {tmp_inputs.shape} (num of examples, length of tweet tensors)")
print(f"The shape of the labels is {tmp_targets.shape}, which is the batch size.")
print(f"The shape of the example_weights is {tmp_example_weights.shape}, which is the same as inputs/targets size.")

The batch is a tuple of length 3 because position 0 contains the tweets, and position 1 contains the targets.
The shape of the tweet tensors is (16, 575) (num of examples, length of tweet tensors)
The shape of the labels is (16,), which is the batch size.
The shape of the example_weights is (16,), which is the same as inputs/targets size.


In [39]:
tmp_pred = training_loop.eval_model(tmp_inputs)
print(f"The prediction shape is {tmp_pred.shape}, num of tensor_tweets as rows")
print("Column 0 is the probability of a negative sentiment (class 0)")
print("Column 1 is the probability of a positive sentiment (class 1)")
print()
print("View the prediction array")
tmp_pred

The prediction shape is (16, 2), num of tensor_tweets as rows
Column 0 is the probability of a negative sentiment (class 0)
Column 1 is the probability of a positive sentiment (class 1)

View the prediction array


DeviceArray([[-7.71493912e-02, -2.60033798e+00],
             [-6.19888306e-06, -1.20189018e+01],
             [-1.21895485e+01, -5.24520874e-06],
             [-1.27900543e+01, -2.86102295e-06],
             [-2.86102295e-06, -1.28113918e+01],
             [-1.22818527e+01, -4.76837158e-06],
             [-2.79283428e+00, -6.32033348e-02],
             [-8.10623169e-06, -1.17256804e+01],
             [-7.84419394e+00, -3.91960144e-04],
             [-1.04904175e-05, -1.14506378e+01],
             [-2.29597092e-04, -8.37881184e+00],
             [-1.90734863e-06, -1.31815567e+01],
             [-2.50172186e+00, -8.54965448e-02],
             [-1.24684887e+01, -3.81469727e-06],
             [-8.16675377e+00, -2.84194946e-04],
             [-1.03420935e+01, -3.24249268e-05]], dtype=float32)

In [40]:
# turn probabilites into category predictions
tmp_is_positive = tmp_pred[:,1] > tmp_pred[:,0]
for i, p in enumerate(tmp_is_positive):
    print(f"Neg log prob {tmp_pred[i,0]:.4f}\tPos log prob {tmp_pred[i,1]:.4f}\t is positive? {p}\t actual {tmp_targets[i]}")

Neg log prob -0.0771	Pos log prob -2.6003	 is positive? False	 actual 0
Neg log prob -0.0000	Pos log prob -12.0189	 is positive? False	 actual 0
Neg log prob -12.1895	Pos log prob -0.0000	 is positive? True	 actual 1
Neg log prob -12.7901	Pos log prob -0.0000	 is positive? True	 actual 1
Neg log prob -0.0000	Pos log prob -12.8114	 is positive? False	 actual 0
Neg log prob -12.2819	Pos log prob -0.0000	 is positive? True	 actual 1
Neg log prob -2.7928	Pos log prob -0.0632	 is positive? True	 actual 1
Neg log prob -0.0000	Pos log prob -11.7257	 is positive? False	 actual 0
Neg log prob -7.8442	Pos log prob -0.0004	 is positive? True	 actual 1
Neg log prob -0.0000	Pos log prob -11.4506	 is positive? False	 actual 0
Neg log prob -0.0002	Pos log prob -8.3788	 is positive? False	 actual 0
Neg log prob -0.0000	Pos log prob -13.1816	 is positive? False	 actual 0
Neg log prob -2.5017	Pos log prob -0.0855	 is positive? True	 actual 0
Neg log prob -12.4685	Pos log prob -0.0000	 is positive? True	

In [41]:
def compute_accuracy(preds, y, y_weights):
    """
    Input: 
        preds: a tensor of shape (dim_batch, output_dim) 
        y: a tensor of shape (dim_batch, output_dim) with the true labels
        y_weights: a n.ndarray with the a weight for each example
    Output: 
        accuracy: a float between 0-1 
        weighted_num_correct (np.float32): Sum of the weighted correct predictions
        sum_weights (np.float32): Sum of the weights
    """
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    # Create an array of booleans, 
    # True if the probability of positive sentiment is greater than
    # the probability of negative sentiment
    # else False
    is_pos = preds[:,1] > preds[:,0]

    # convert the array of booleans into an array of np.int32
    is_pos_int = is_pos.astype(np.int32)
    
    # compare the array of predictions (as int32) with the target (labels) of type int32
    correct = is_pos_int == np.array(y)

    # Count the sum of the weights.
    sum_weights = np.sum(y_weights)
    
    # convert the array of correct predictions (boolean) into an arrayof np.float32
    correct_float = correct.astype(np.float32)
    
    # Multiply each prediction with its corresponding weight.
    weighted_correct_float = correct_float * np.array(y_weights)

    # Sum up the weighted correct predictions (of type np.float32), to go in the
    # denominator.
    weighted_num_correct = np.sum(weighted_correct_float)
 
    # Divide the number of weighted correct predictions by the sum of the
    # weights.
    accuracy = weighted_num_correct / sum_weights

    ### END CODE HERE ###
    return accuracy, weighted_num_correct, sum_weights

In [42]:
# test your function
tmp_val_generator = val_generator(64)

# get one batch
tmp_batch = next(tmp_val_generator)

# Position 0 has the model inputs (tweets as tensors)
# position 1 has the targets (the actual labels)
tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

# feed the tweet tensors into the model to get a prediction
tmp_pred = training_loop.eval_model(tmp_inputs)

tmp_acc, tmp_num_correct, tmp_num_predictions = compute_accuracy(preds=tmp_pred, y=tmp_targets, y_weights=tmp_example_weights)

print(f"Model's prediction accuracy on a single training batch is: {100 * tmp_acc}%")
print(f"Weighted number of correct predictions {tmp_num_correct}; weighted number of total observations predicted {tmp_num_predictions}")

Model's prediction accuracy on a single training batch is: 95.3125%
Weighted number of correct predictions 61.0; weighted number of total observations predicted 64


In [43]:
def test_model(generator, model):
    '''
    Input: 
        generator: an iterator instance that provides batches of inputs and targets
        model: a model instance 
    Output: 
        accuracy: float corresponding to the accuracy
    '''
    
    accuracy = 0.
    total_num_correct = 0
    total_num_pred = 0
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    for batch in generator: 
        
        # Retrieve the inputs from the batch
        inputs = batch[0]
        
        # Retrieve the targets (actual labels) from the batch
        targets = batch[1]
        
        # Retrieve the example weight.
        example_weight = batch[2]

        # Make predictions using the inputs
        pred = model(inputs)
        
        # Calculate accuracy for the batch by comparing its predictions and targets
        batch_accuracy, batch_num_correct, batch_num_pred = compute_accuracy(preds=pred, y=targets, y_weights=example_weight)
        
        # Update the total number of correct predictions
        # by adding the number of correct predictions from this batch
        total_num_correct += batch_num_correct
        
        # Update the total number of predictions 
        # by adding the number of predictions made for the batch
        total_num_pred += batch_num_pred

    # Calculate accuracy over all examples
    accuracy = total_num_correct/total_num_pred
    
    ### END CODE HERE ###
    return accuracy

In [44]:
model = training_loop.eval_model
accuracy = test_model(test_generator(16), model)

print(f'The accuracy of your model on the validation set is {accuracy:.4f}', )

The accuracy of your model on the validation set is 0.9427


In [45]:
def predict(sentence):
    inputs = np.array(text_to_tensor(sentence, vocab_dict=Vocab))
    
    # Batch size 1, add dimension for batch, to work with the model
    inputs = inputs[None, :]  
    
    # predict with the model
    preds_probs = model(inputs)
    
    # Turn probabilities into categories
    preds = int(preds_probs[0, 1] > preds_probs[0, 0])
    
    sen_pred = "Not Fake"
    if preds == 1:
        sen_pred = 'Fake'
        
    print("Fake ",str(preds_probs[0, 1]), " Not Fake ",str(preds_probs[0, 0]))

    return preds, sen_pred

In [46]:
sentence = "Donald John Trump (born June 14, 1946) is the 45th and current president of the United States. Before entering politics, he was a businessman and television personality. Trump was born and raised in Queens, a borough of New York City. He attended Fordham University for two years and received a bachelor's degree in economics from the Wharton School of the University of Pennsylvania. He became president of his father's real-estate business in 1971, renamed it The Trump Organization, and expanded its operations to building or renovating skyscrapers, hotels, casinos, and golf courses. Trump later started various side ventures, mostly by licensing his name. Trump and his businesses have been involved in more than 4,000 state and federal legal actions, including six bankruptcies. He owned the Miss Universe brand of beauty pageants from 1996 to 2015. He produced and hosted The Apprentice, a reality television series, from 2003 to 2015. As of April 2020, Forbes estimated his net worth to be $2.1 billion."
tmp_pred, tmp_sen_pred = predict(sentence)
print(f"The sentence \n***\n\"{sentence}\"\n***\nis {tmp_sen_pred}.")

Fake  -7.9701853  Not Fake  -0.00034570694
The sentence 
***
"Donald John Trump (born June 14, 1946) is the 45th and current president of the United States. Before entering politics, he was a businessman and television personality. Trump was born and raised in Queens, a borough of New York City. He attended Fordham University for two years and received a bachelor's degree in economics from the Wharton School of the University of Pennsylvania. He became president of his father's real-estate business in 1971, renamed it The Trump Organization, and expanded its operations to building or renovating skyscrapers, hotels, casinos, and golf courses. Trump later started various side ventures, mostly by licensing his name. Trump and his businesses have been involved in more than 4,000 state and federal legal actions, including six bankruptcies. He owned the Miss Universe brand of beauty pageants from 1996 to 2015. He produced and hosted The Apprentice, a reality television series, from 2003 to 2

In [47]:
sentence = all_neg_text[-1]
tmp_pred, tmp_sen_pred = predict(sentence)
print(f"The sentence \n***\n\"{sentence}\"\n***\nis {tmp_sen_pred}.")

Fake  -12.409691  Not Fake  -3.8146973e-06
The sentence 
***
"The Macy’s of today grew from the union of several great names in American retailing, including its namesake chain, Bloomingdale’s and Marshall Field’s. But the ambitious owner of Saks Fifth Avenue has broached the idea of taking the union even further, combining with Macy’s to create a department store juggernaut at a time when the industry is reeling. Hudson’s Bay Company, the Canadian owner of Saks, has approached Macy’s about a potential takeover, people briefed on the matter who were not authorized to speak publicly said on Friday. Talks between the two companies are at an early stage and may still fall apart or lead to a partnership of some kind rather than a sale. While it is unclear whether a deal will happen, a combination could lift the fortunes of Macy’s, the country’s biggest department store, which has been struggling. Investors certainly appeared to see it that way. Shares of Macy’s rose as much as 12 percent o

In [48]:
sentence = all_pos_text[-1]
tmp_pred, tmp_sen_pred = predict(sentence)
print(f"The sentence \n***\n\"{sentence}\"\n***\nis {tmp_sen_pred}.")

Fake  -1.7642975e-05  Not Fake  -10.937075
The sentence 
***
"  David Swanson is an author, activist, journalist, and radio host. He is a 2015 Nobel Peace Prize Nominee. He is director of WorldBeyondWar.com and campaign coordinator for RootsAction.org . He hosts Talk Nation Radio . Talk Nation Radio is on VT Radio and is syndicated by Pacifica Network. The show also airs on WTJU, Charlottesville, VA; WCSX-Detroit, MI; KGHI, Westport, WA; WHUS, Storrs, CT; WPRR, Grand Rapids, MI; KRFP-LP, Moscow, ID; KZGM, Cabool, MO; KMUD, Garberville, CA; WAZU, Peoria, IL; WXRD, Crown Point, IN; Geneva Radio, Geneva, NY; KKRN, Round Mountain, CA; KSKQ-LP, Ashland, OR; WUOW-LP, Oneonta, NY; No Lies Radio, Pinole, CA; WYAP-LP, Clay, WV; The Detour, Johnson City, TN; WZRD, Chicago, IL; WEFT, Champaign, IL; WXPI, Pittsburgh, PA; WDRT, Viroqua, WI; Veracity Now, online; Liberty and Justice Radio, Shirley, MA; Ithaca Community Radio, Ithaca, NY; WMCB, Greenfield, MA; PRX.org; KAOS 89.3fm, Olympia, WA; WUSB 

In [49]:
sentence = "The first known use of the name \"America\" dates back to 1507, when it appeared on a world map created by the German cartographer Martin Waldseemüller. It comes from the surname of Italian explorer Amerigo Vespucci, who was the first to postulate that the West Indies did not represent Asia's eastern limit, but part of a previously unknown landmass.[20][21] The full name \"United States of America\" dates to the Revolutionary War. The second draft of the Articles of Confederation, prepared by John Dickinson, and Thomas Jefferson's original rough draft of the Declaration of Independence both included the name. However, it is unclear which of these two usages came first"
tmp_pred, tmp_sen_pred = predict(sentence)
print(f"The sentence \n***\n\"{sentence}\"\n***\nis {tmp_sen_pred}.")

Fake  -4.7683716e-06  Not Fake  -12.244802
The sentence 
***
"The first known use of the name "America" dates back to 1507, when it appeared on a world map created by the German cartographer Martin Waldseemüller. It comes from the surname of Italian explorer Amerigo Vespucci, who was the first to postulate that the West Indies did not represent Asia's eastern limit, but part of a previously unknown landmass.[20][21] The full name "United States of America" dates to the Revolutionary War. The second draft of the Articles of Confederation, prepared by John Dickinson, and Thomas Jefferson's original rough draft of the Declaration of Independence both included the name. However, it is unclear which of these two usages came first"
***
is Fake.
